In [3]:
import random
!pip install gym


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import os

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [4]:
import sys


# Reduce Recursion limit
sys.setrecursionlimit(1500)
import gym
import numpy as np

# Create the environment
env = gym.make('CartPole-v1')

# Set random seed for reproducibility
np.random.seed(42)
env.action_space.seed(42)
env.observation_space.seed(42)

[42]

In [7]:
import warnings
warnings.filterwarnings('ignore')

# Override the default warning function
def warn(*args, **kwargs):
    pass
warnings.warn = warn

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

def build_model(state_size, action_size):
    model = Sequential()
    model.add(Input(shape=(state_size,))) # Use Input layer to specify the input shape
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(action_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
    return model

# Create the environment and set up the model
env = gym.make('CartPole-v1')
state_size_ = env.observation_space.shape[0]
action_size_ = env.action_space.n
model_ = build_model(state_size_, action_size_)

In [11]:
from collections import deque
import random
import tensorflow as tf

# Define epsilon and epsilon_decay
epsilon = 1. # Starting with a high exploration rate
epsilon_min = .01 # Minium exploration rate
epsilon_decay = 0.99 # Faster decay rate for epsilon after each episode

# Replay memory
memory = deque(maxlen=2000)

def remember(state, action, reward, next_state, done):
    """Store experience in memory."""
    memory.append((state, action, reward, next_state, done))

def replay(batch_size=64):
    """Train the model using a random sample of experiences from memory."""
    if len(memory) < batch_size:
        return # Skip replay if there's not enough experience

    minibatch = random.sample(memory, batch_size) # Sample a random batch from memory

    # Extract information for batch processing
    states = np.vstack([x[0] for x in minibatch])
    actions = np.array([x[1] for x in minibatch])
    rewards = np.array([x[2] for x in minibatch])
    next_states = np.vstack([x[3] for x in minibatch])
    dones = np.array([x[4] for x in minibatch])

    # Predict Q-values for the next states in batch
    q_next = model_.predict(next_states)
    # Predict Q-values for the current states in batch
    q_target = model_.predict(states)

    # Vectorized update of target values
    for i in range(batch_size):
        target = rewards[i]
        if not dones[i]:
            target += .95 * np.amax(q_next[i]) # Update Q value with the discounted future rewed
        q_target[i][actions[i]] = target # update only the take action's Q value

    # Train the model with the updated targets in batch
    model_.fit(states, q_target, epochs=1, verbose=0) # Train in batch mode

    # Reduce exploration rate (epsilon) after each training step
    global epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

def act(state):
    """Choose an action based on the current state and exploration rate."""
    if np.random.rand() <= epsilon:
        return random.randrange(action_size_) # Explore: choose a random action
    act_values = model_.predict(state) # Exploit: predict action based on the state
    return np.argmax(act_values) # Return the action with the highest Q-value

# Define the number of episodes you want to train the model for
episodes = 10
train_frequency = 5 # Train the model every 5 steps

for e in range(episodes):
    state_, _ = env.reset() # Unpack the tuple returned by env.reset()
    state_ = np.reshape(state_, [1, state_size_])
    for time in range(200): # Limit to 200 time steps per episode
        action_ = act(state_)
        next_state_, reward_, terminated, truncated, _ = env.step(action_)
        done_ = terminated or truncated
        reward_ = reward_ if not done_ else -10
        next_state_ = np.reshape(next_state_, [1, state_size_])
        remember(state_, action_, reward_, next_state_, done_) # Store experience
        state_ = next_state_

        if done_:
            print(f"Episode: {e+1}/{episodes}, score: {time}, e: {epsilon:.2}")
            break
        # Train the model every 'train_frequency' steps
        if time % train_frequency == 0:
            replay(batch_size=64) # Call replay with larger batch size for efficiency

env.close()

Episode: 1/10, score: 19, e: 1.0
Episode: 2/10, score: 13, e: 1.0
Episode: 3/10, score: 16, e: 1.0
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Episode: 4/10, score: 23, e: 0.98
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Episode: 5/10, score: 15, e: 0.95
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/st

### Evaluate the Performance

In [12]:
for e in range(10):
    state_, _ = env.reset() # Unpack the state from the tuple
    state_ = np.reshape(state_, [1, state_size_]) # Reshape the state correctly
    for time in range(500):
        env.render()
        action_ = np.argmax(model_.predict(state_)[0])
        next_state_, reward_, terminated, truncated, _ = env.step(action_) # Unpack the five return values
        done_ = terminated or truncated # Check if the episode is done
        next_state_ = np.reshape(next_state_, [1, state_size_])
        state_ = next_state_
        if done_:
            print(f"Episode: {e+1}/10, score: {time}, e: {epsilon:.2}")
            break

env.close()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Episode: 1/10, score: 9, e: 0.76
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Episode: 2/10, score: 9, e: 0.76
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━